# Demo de Transformers disponibles en HuggingFace para procesamiento de Texto

https://huggingface.co/models

In [1]:
#@title Instalar paquete Transformers de HuggingFace
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.3 MB/s eta 0:00:00


In [2]:
#@title Instalar otras librerías

!pip install fairseq

!pip install sentencepiece

!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 50.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.6/269.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp39-cp39-linux_x86_64.whl size=11180033 sha256=03c64fea3011ca356fe7e3f160760a6fd725b28343ec59f2e328db62db708d3f
  Stored in directory: /root/.cache/pip/wheels/59/35/87/2baf2e4ad37c83fd698c486b3d39f0e7022226fa52ab469c31
  Crea

In [3]:
#@title Cargar Librerías

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoModelForSeq2SeqLM, pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch  # nota: usa Torch porque con TF directo tira error
import re
import random

from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
from fairseq import utils

#import IPython.display as ipd
from IPython.display import Audio, display
import textwrap

print("Librerías cargadas.")

# determina si usa GPU o CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Device: ", device)

Librerías cargadas.
Device:  cpu


# Modelo para Responder Preguntas ( Q&A )

In [4]:
#@title Cargar Modelo Q&A 

nombreModeloQA = "timpal0l/mdeberta-v3-base-squad2" #@param [ "timpal0l/mdeberta-v3-base-squad2", "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es", "BSC-LT/roberta-large-bne-sqac"]

# cargar modelo
tokenizerQA = AutoTokenizer.from_pretrained(nombreModeloQA)
modelQA = AutoModelForQuestionAnswering.from_pretrained(nombreModeloQA)

# función para ejecutar el modelo
def responder_pregunta(question, context):
  # prepara input
  inputs = tokenizerQA(question, context, return_tensors="pt")
  with torch.no_grad():
      outputs = modelQA(**inputs)
  # detrmina rango
  answer_start_index = torch.argmax(outputs.start_logits)
  answer_end_index = torch.argmax(outputs.end_logits)
  # ejecuta el modelo y decodifica
  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  return tokenizerQA.decode(predict_answer_tokens)

print("\nModelo ", nombreModeloQA, " cargado.")


Modelo  timpal0l/mdeberta-v3-base-squad2  cargado.


In [5]:
#@title Prueba 1 modelo Q&A

context = "San Martín cabalgaba en su caballo blanco cuando ataco al enemigo." #@param {type:"string"}
question = "de que color es caballo San Martín?" #@param {type:"string"}

# procesa pregunta por modelo
res = responder_pregunta(question, context)

print("\n")
print("> ", textwrap.fill(context))
print("\t", question)
print("\t  -->", res)




>  San Martín cabalgaba en su caballo blanco cuando ataco al enemigo.
	 de que color es caballo San Martín?
	  --> blanco


In [6]:
#@title Prueba 2 modelo Q&A

context = "Luego de saltar sobre la pelota roja, el perro gris cay\xF3 dentro de la pileta" #@param {type:"string"}

question_1 = "que color es perro?" #@param {type:"string"}

question_2 = "que color es pelota?" #@param {type:"string"}

question_3 = "que hizo el perro?" #@param {type:"string"}

question_4 = "donde salto?" #@param {type:"string"}

question_5 = "donde termino?" #@param {type:"string"}


questionList = [question_1, question_2, question_3, question_4, question_5] 


print("\n")
print("> ", textwrap.fill(context, 100))

for q in questionList:
  # procesa pregunta por modelo
  res = responder_pregunta(q, context)

  print("\t", q)
  print("\t  -->", res)
  print("")




>  Luego de saltar sobre la pelota roja, el perro gris cayó dentro de la pileta
	 que color es perro?
	  --> gris

	 que color es pelota?
	  --> roja

	 que hizo el perro?
	  --> saltar sobre la pelota roja, el perro gris cayó dentro de la pileta

	 donde salto?
	  --> sobre la pelota roja

	 donde termino?
	  --> dentro de la pileta



# Modelo Traductor

In [7]:
#@title Cargar Modelo Traductor

nombreModeloTrad = "facebook/nllb-200-distilled-600M"

tokenizerTrad = AutoTokenizer.from_pretrained(nombreModeloTrad)
modelTrad = AutoModelForSeq2SeqLM.from_pretrained(nombreModeloTrad)

def translate(text, src_lang, tgt_lang, max_length=400):

    TASK = "translation"
    translation_pipeline = pipeline(TASK,
                                    model=modelTrad,
                                    tokenizer=tokenizerTrad,
                                    src_lang=src_lang,
                                    tgt_lang=tgt_lang,
                                    max_length=max_length,
                                    device=device)

    result = translation_pipeline(text)
    return result[0]['translation_text']

print("\nModelo ", nombreModeloTrad, " cargado.")    


Modelo  facebook/nllb-200-distilled-600M  cargado.


In [8]:
#@title Probar Traductor 1

lenguaje_texto = "eng_Latn" #@param ["ace_Arab",  "ace_Latn",  "acm_Arab",  "acq_Arab",  "aeb_Arab",  "afr_Latn",  "ajp_Arab",  "aka_Latn",  "amh_Ethi",  "apc_Arab",  "arb_Arab",  "ars_Arab",  "ary_Arab",  "arz_Arab",  "asm_Beng",  "ast_Latn",  "awa_Deva",  "ayr_Latn",  "azb_Arab",  "azj_Latn",  "bak_Cyrl",  "bam_Latn",  "ban_Latn",  "bel_Cyrl",  "bem_Latn",  "ben_Beng",  "bho_Deva",  "bjn_Arab",  "bjn_Latn",  "bod_Tibt",  "bos_Latn",  "bug_Latn",  "bul_Cyrl",  "cat_Latn",  "ceb_Latn",  "ces_Latn",  "cjk_Latn",  "ckb_Arab",  "crh_Latn",  "cym_Latn",  "dan_Latn",  "deu_Latn",  "dik_Latn",  "dyu_Latn",  "dzo_Tibt",  "ell_Grek",  "eng_Latn",  "epo_Latn",  "est_Latn",  "eus_Latn",  "ewe_Latn",  "fao_Latn",  "pes_Arab",  "fij_Latn",  "fin_Latn",  "fon_Latn",  "fra_Latn",  "fur_Latn",  "fuv_Latn",  "gla_Latn",  "gle_Latn",  "glg_Latn",  "grn_Latn",  "guj_Gujr",  "hat_Latn",  "hau_Latn",  "heb_Hebr",  "hin_Deva",  "hne_Deva",  "hrv_Latn",  "hun_Latn",  "hye_Armn",  "ibo_Latn",  "ilo_Latn",  "ind_Latn",  "isl_Latn",  "ita_Latn",  "jav_Latn",  "jpn_Jpan",  "kab_Latn",  "kac_Latn",  "kam_Latn",  "kan_Knda",  "kas_Arab",  "kas_Deva",  "kat_Geor",  "knc_Arab",  "knc_Latn",  "kaz_Cyrl",  "kbp_Latn",  "kea_Latn",  "khm_Khmr",  "kik_Latn",  "kin_Latn",  "kir_Cyrl",  "kmb_Latn",  "kon_Latn",  "kor_Hang",  "kmr_Latn",  "lao_Laoo",  "lvs_Latn",  "lij_Latn",  "lim_Latn",  "lin_Latn",  "lit_Latn",  "lmo_Latn",  "ltg_Latn",  "ltz_Latn",  "lua_Latn",  "lug_Latn",  "luo_Latn",  "lus_Latn",  "mag_Deva",  "mai_Deva",  "mal_Mlym",  "mar_Deva",  "min_Latn",  "mkd_Cyrl",  "plt_Latn",  "mlt_Latn",  "mni_Beng",  "khk_Cyrl",  "mos_Latn",  "mri_Latn",  "zsm_Latn",  "mya_Mymr",  "nld_Latn",  "nno_Latn",  "nob_Latn",  "npi_Deva",  "nso_Latn",  "nus_Latn",  "nya_Latn",  "oci_Latn",  "gaz_Latn",  "ory_Orya",  "pag_Latn",  "pan_Guru",  "pap_Latn",  "pol_Latn",  "por_Latn",  "prs_Arab",  "pbt_Arab",  "quy_Latn",  "ron_Latn",  "run_Latn",  "rus_Cyrl",  "sag_Latn",  "san_Deva",  "sat_Beng",  "scn_Latn",  "shn_Mymr",  "sin_Sinh",  "slk_Latn",  "slv_Latn",  "smo_Latn",  "sna_Latn",  "snd_Arab",  "som_Latn",  "sot_Latn",  "spa_Latn",  "als_Latn",  "srd_Latn",  "srp_Cyrl",  "ssw_Latn",  "sun_Latn",  "swe_Latn",  "swh_Latn",  "szl_Latn",  "tam_Taml",  "tat_Cyrl",  "tel_Telu",  "tgk_Cyrl",  "tgl_Latn",  "tha_Thai",  "tir_Ethi",  "taq_Latn",  "taq_Tfng",  "tpi_Latn",  "tsn_Latn",  "tso_Latn",  "tuk_Latn",  "tum_Latn",  "tur_Latn",  "twi_Latn",  "tzm_Tfng",  "uig_Arab",  "ukr_Cyrl",  "umb_Latn",  "urd_Arab",  "uzn_Latn",  "vec_Latn",  "vie_Latn",  "war_Latn",  "wol_Latn",  "xho_Latn",  "ydd_Hebr",  "yor_Latn",  "yue_Hant",  "zho_Hans",  "zho_Hant",  "zul_Latn" ]

text_to_translate = "Life is like a box of chocolates" #@param {type:"string"}

lenguaje_traduccion = "spa_Latn" #@param ["ace_Arab",  "ace_Latn",  "acm_Arab",  "acq_Arab",  "aeb_Arab",  "afr_Latn",  "ajp_Arab",  "aka_Latn",  "amh_Ethi",  "apc_Arab",  "arb_Arab",  "ars_Arab",  "ary_Arab",  "arz_Arab",  "asm_Beng",  "ast_Latn",  "awa_Deva",  "ayr_Latn",  "azb_Arab",  "azj_Latn",  "bak_Cyrl",  "bam_Latn",  "ban_Latn",  "bel_Cyrl",  "bem_Latn",  "ben_Beng",  "bho_Deva",  "bjn_Arab",  "bjn_Latn",  "bod_Tibt",  "bos_Latn",  "bug_Latn",  "bul_Cyrl",  "cat_Latn",  "ceb_Latn",  "ces_Latn",  "cjk_Latn",  "ckb_Arab",  "crh_Latn",  "cym_Latn",  "dan_Latn",  "deu_Latn",  "dik_Latn",  "dyu_Latn",  "dzo_Tibt",  "ell_Grek",  "eng_Latn",  "epo_Latn",  "est_Latn",  "eus_Latn",  "ewe_Latn",  "fao_Latn",  "pes_Arab",  "fij_Latn",  "fin_Latn",  "fon_Latn",  "fra_Latn",  "fur_Latn",  "fuv_Latn",  "gla_Latn",  "gle_Latn",  "glg_Latn",  "grn_Latn",  "guj_Gujr",  "hat_Latn",  "hau_Latn",  "heb_Hebr",  "hin_Deva",  "hne_Deva",  "hrv_Latn",  "hun_Latn",  "hye_Armn",  "ibo_Latn",  "ilo_Latn",  "ind_Latn",  "isl_Latn",  "ita_Latn",  "jav_Latn",  "jpn_Jpan",  "kab_Latn",  "kac_Latn",  "kam_Latn",  "kan_Knda",  "kas_Arab",  "kas_Deva",  "kat_Geor",  "knc_Arab",  "knc_Latn",  "kaz_Cyrl",  "kbp_Latn",  "kea_Latn",  "khm_Khmr",  "kik_Latn",  "kin_Latn",  "kir_Cyrl",  "kmb_Latn",  "kon_Latn",  "kor_Hang",  "kmr_Latn",  "lao_Laoo",  "lvs_Latn",  "lij_Latn",  "lim_Latn",  "lin_Latn",  "lit_Latn",  "lmo_Latn",  "ltg_Latn",  "ltz_Latn",  "lua_Latn",  "lug_Latn",  "luo_Latn",  "lus_Latn",  "mag_Deva",  "mai_Deva",  "mal_Mlym",  "mar_Deva",  "min_Latn",  "mkd_Cyrl",  "plt_Latn",  "mlt_Latn",  "mni_Beng",  "khk_Cyrl",  "mos_Latn",  "mri_Latn",  "zsm_Latn",  "mya_Mymr",  "nld_Latn",  "nno_Latn",  "nob_Latn",  "npi_Deva",  "nso_Latn",  "nus_Latn",  "nya_Latn",  "oci_Latn",  "gaz_Latn",  "ory_Orya",  "pag_Latn",  "pan_Guru",  "pap_Latn",  "pol_Latn",  "por_Latn",  "prs_Arab",  "pbt_Arab",  "quy_Latn",  "ron_Latn",  "run_Latn",  "rus_Cyrl",  "sag_Latn",  "san_Deva",  "sat_Beng",  "scn_Latn",  "shn_Mymr",  "sin_Sinh",  "slk_Latn",  "slv_Latn",  "smo_Latn",  "sna_Latn",  "snd_Arab",  "som_Latn",  "sot_Latn",  "spa_Latn",  "als_Latn",  "srd_Latn",  "srp_Cyrl",  "ssw_Latn",  "sun_Latn",  "swe_Latn",  "swh_Latn",  "szl_Latn",  "tam_Taml",  "tat_Cyrl",  "tel_Telu",  "tgk_Cyrl",  "tgl_Latn",  "tha_Thai",  "tir_Ethi",  "taq_Latn",  "taq_Tfng",  "tpi_Latn",  "tsn_Latn",  "tso_Latn",  "tuk_Latn",  "tum_Latn",  "tur_Latn",  "twi_Latn",  "tzm_Tfng",  "uig_Arab",  "ukr_Cyrl",  "umb_Latn",  "urd_Arab",  "uzn_Latn",  "vec_Latn",  "vie_Latn",  "war_Latn",  "wol_Latn",  "xho_Latn",  "ydd_Hebr",  "yor_Latn",  "yue_Hant",  "zho_Hans",  "zho_Hant",  "zul_Latn" ]

# traduce texto
resTrad = translate(text_to_translate, lenguaje_texto, lenguaje_traduccion)

print("\n", text_to_translate)
print("\t--> ", resTrad)

# traduce al revés (para probar)
resTrad2 = translate(resTrad, lenguaje_traduccion, lenguaje_texto)

print("\n", resTrad)
print("\t--> ", resTrad2)



 Life is like a box of chocolates
	-->  La vida es como una caja de chocolate

 La vida es como una caja de chocolate
	-->  Life is like a box of chocolate


In [9]:
#@title Probar Traductor 2

lenguaje_texto = "spa_Latn" #@param ["ace_Arab",  "ace_Latn",  "acm_Arab",  "acq_Arab",  "aeb_Arab",  "afr_Latn",  "ajp_Arab",  "aka_Latn",  "amh_Ethi",  "apc_Arab",  "arb_Arab",  "ars_Arab",  "ary_Arab",  "arz_Arab",  "asm_Beng",  "ast_Latn",  "awa_Deva",  "ayr_Latn",  "azb_Arab",  "azj_Latn",  "bak_Cyrl",  "bam_Latn",  "ban_Latn",  "bel_Cyrl",  "bem_Latn",  "ben_Beng",  "bho_Deva",  "bjn_Arab",  "bjn_Latn",  "bod_Tibt",  "bos_Latn",  "bug_Latn",  "bul_Cyrl",  "cat_Latn",  "ceb_Latn",  "ces_Latn",  "cjk_Latn",  "ckb_Arab",  "crh_Latn",  "cym_Latn",  "dan_Latn",  "deu_Latn",  "dik_Latn",  "dyu_Latn",  "dzo_Tibt",  "ell_Grek",  "eng_Latn",  "epo_Latn",  "est_Latn",  "eus_Latn",  "ewe_Latn",  "fao_Latn",  "pes_Arab",  "fij_Latn",  "fin_Latn",  "fon_Latn",  "fra_Latn",  "fur_Latn",  "fuv_Latn",  "gla_Latn",  "gle_Latn",  "glg_Latn",  "grn_Latn",  "guj_Gujr",  "hat_Latn",  "hau_Latn",  "heb_Hebr",  "hin_Deva",  "hne_Deva",  "hrv_Latn",  "hun_Latn",  "hye_Armn",  "ibo_Latn",  "ilo_Latn",  "ind_Latn",  "isl_Latn",  "ita_Latn",  "jav_Latn",  "jpn_Jpan",  "kab_Latn",  "kac_Latn",  "kam_Latn",  "kan_Knda",  "kas_Arab",  "kas_Deva",  "kat_Geor",  "knc_Arab",  "knc_Latn",  "kaz_Cyrl",  "kbp_Latn",  "kea_Latn",  "khm_Khmr",  "kik_Latn",  "kin_Latn",  "kir_Cyrl",  "kmb_Latn",  "kon_Latn",  "kor_Hang",  "kmr_Latn",  "lao_Laoo",  "lvs_Latn",  "lij_Latn",  "lim_Latn",  "lin_Latn",  "lit_Latn",  "lmo_Latn",  "ltg_Latn",  "ltz_Latn",  "lua_Latn",  "lug_Latn",  "luo_Latn",  "lus_Latn",  "mag_Deva",  "mai_Deva",  "mal_Mlym",  "mar_Deva",  "min_Latn",  "mkd_Cyrl",  "plt_Latn",  "mlt_Latn",  "mni_Beng",  "khk_Cyrl",  "mos_Latn",  "mri_Latn",  "zsm_Latn",  "mya_Mymr",  "nld_Latn",  "nno_Latn",  "nob_Latn",  "npi_Deva",  "nso_Latn",  "nus_Latn",  "nya_Latn",  "oci_Latn",  "gaz_Latn",  "ory_Orya",  "pag_Latn",  "pan_Guru",  "pap_Latn",  "pol_Latn",  "por_Latn",  "prs_Arab",  "pbt_Arab",  "quy_Latn",  "ron_Latn",  "run_Latn",  "rus_Cyrl",  "sag_Latn",  "san_Deva",  "sat_Beng",  "scn_Latn",  "shn_Mymr",  "sin_Sinh",  "slk_Latn",  "slv_Latn",  "smo_Latn",  "sna_Latn",  "snd_Arab",  "som_Latn",  "sot_Latn",  "spa_Latn",  "als_Latn",  "srd_Latn",  "srp_Cyrl",  "ssw_Latn",  "sun_Latn",  "swe_Latn",  "swh_Latn",  "szl_Latn",  "tam_Taml",  "tat_Cyrl",  "tel_Telu",  "tgk_Cyrl",  "tgl_Latn",  "tha_Thai",  "tir_Ethi",  "taq_Latn",  "taq_Tfng",  "tpi_Latn",  "tsn_Latn",  "tso_Latn",  "tuk_Latn",  "tum_Latn",  "tur_Latn",  "twi_Latn",  "tzm_Tfng",  "uig_Arab",  "ukr_Cyrl",  "umb_Latn",  "urd_Arab",  "uzn_Latn",  "vec_Latn",  "vie_Latn",  "war_Latn",  "wol_Latn",  "xho_Latn",  "ydd_Hebr",  "yor_Latn",  "yue_Hant",  "zho_Hans",  "zho_Hant",  "zul_Latn" ]

text_to_translate = "Que lindo día para un asado!" #@param {type:"string"}

cant_idiomas_usar = 10 #@param {type:"integer"}

if cant_idiomas_usar < 2:
  cant_idiomas_usar = 2

print("\n", text_to_translate)

lista_idiomas = ["ace_Arab",  "ace_Latn",  "acm_Arab",  "acq_Arab",  "aeb_Arab",  "afr_Latn",  "ajp_Arab",  "aka_Latn",  "amh_Ethi",  "apc_Arab",  "arb_Arab",  "ars_Arab",  "ary_Arab",  "arz_Arab",  "asm_Beng",  "ast_Latn",  "awa_Deva",  "ayr_Latn",  "azb_Arab",  "azj_Latn",  "bak_Cyrl",  "bam_Latn",  "ban_Latn",  "bel_Cyrl",  "bem_Latn",  "ben_Beng",  "bho_Deva",  "bjn_Arab",  "bjn_Latn",  "bod_Tibt",  "bos_Latn",  "bug_Latn",  "bul_Cyrl",  "cat_Latn",  "ceb_Latn",  "ces_Latn",  "cjk_Latn",  "ckb_Arab",  "crh_Latn",  "cym_Latn",  "dan_Latn",  "deu_Latn",  "dik_Latn",  "dyu_Latn",  "dzo_Tibt",  "ell_Grek",  "eng_Latn",  "epo_Latn",  "est_Latn",  "eus_Latn",  "ewe_Latn",  "fao_Latn",  "pes_Arab",  "fij_Latn",  "fin_Latn",  "fon_Latn",  "fra_Latn",  "fur_Latn",  "fuv_Latn",  "gla_Latn",  "gle_Latn",  "glg_Latn",  "grn_Latn",  "guj_Gujr",  "hat_Latn",  "hau_Latn",  "heb_Hebr",  "hin_Deva",  "hne_Deva",  "hrv_Latn",  "hun_Latn",  "hye_Armn",  "ibo_Latn",  "ilo_Latn",  "ind_Latn",  "isl_Latn",  "ita_Latn",  "jav_Latn",  "jpn_Jpan",  "kab_Latn",  "kac_Latn",  "kam_Latn",  "kan_Knda",  "kas_Arab",  "kas_Deva",  "kat_Geor",  "knc_Arab",  "knc_Latn",  "kaz_Cyrl",  "kbp_Latn",  "kea_Latn",  "khm_Khmr",  "kik_Latn",  "kin_Latn",  "kir_Cyrl",  "kmb_Latn",  "kon_Latn",  "kor_Hang",  "kmr_Latn",  "lao_Laoo",  "lvs_Latn",  "lij_Latn",  "lim_Latn",  "lin_Latn",  "lit_Latn",  "lmo_Latn",  "ltg_Latn",  "ltz_Latn",  "lua_Latn",  "lug_Latn",  "luo_Latn",  "lus_Latn",  "mag_Deva",  "mai_Deva",  "mal_Mlym",  "mar_Deva",  "min_Latn",  "mkd_Cyrl",  "plt_Latn",  "mlt_Latn",  "mni_Beng",  "khk_Cyrl",  "mos_Latn",  "mri_Latn",  "zsm_Latn",  "mya_Mymr",  "nld_Latn",  "nno_Latn",  "nob_Latn",  "npi_Deva",  "nso_Latn",  "nus_Latn",  "nya_Latn",  "oci_Latn",  "gaz_Latn",  "ory_Orya",  "pag_Latn",  "pan_Guru",  "pap_Latn",  "pol_Latn",  "por_Latn",  "prs_Arab",  "pbt_Arab",  "quy_Latn",  "ron_Latn",  "run_Latn",  "rus_Cyrl",  "sag_Latn",  "san_Deva",  "sat_Beng",  "scn_Latn",  "shn_Mymr",  "sin_Sinh",  "slk_Latn",  "slv_Latn",  "smo_Latn",  "sna_Latn",  "snd_Arab",  "som_Latn",  "sot_Latn",  "spa_Latn",  "als_Latn",  "srd_Latn",  "srp_Cyrl",  "ssw_Latn",  "sun_Latn",  "swe_Latn",  "swh_Latn",  "szl_Latn",  "tam_Taml",  "tat_Cyrl",  "tel_Telu",  "tgk_Cyrl",  "tgl_Latn",  "tha_Thai",  "tir_Ethi",  "taq_Latn",  "taq_Tfng",  "tpi_Latn",  "tsn_Latn",  "tso_Latn",  "tuk_Latn",  "tum_Latn",  "tur_Latn",  "twi_Latn",  "tzm_Tfng",  "uig_Arab",  "ukr_Cyrl",  "umb_Latn",  "urd_Arab",  "uzn_Latn",  "vec_Latn",  "vie_Latn",  "war_Latn",  "wol_Latn",  "xho_Latn",  "ydd_Hebr",  "yor_Latn",  "yue_Hant",  "zho_Hans",  "zho_Hant",  "zul_Latn" ]

lng_random = random.sample(lista_idiomas,  cant_idiomas_usar)

for lng_trg in lng_random:

  # traduce texto
  resTrad = translate(text_to_translate, lenguaje_texto, lng_trg)
  print("\t[", lng_trg,"] --> ", resTrad)



 Que lindo día para un asado!
	[ gla_Latn ] -->  Dè latha brèagha airson brògan!
	[ smo_Latn ] -->  O se aso matagofie mo se falaoa!
	[ ajp_Arab ] -->  يوم جميل لخبز!
	[ spa_Latn ] -->  ¡Qué hermoso día para un asado!
	[ hne_Deva ] -->  एक बेरा बर कइसन सुघर दिन!
	[ ayr_Latn ] -->  ¡Kunja suma urüki uka asado manq'a!
	[ fin_Latn ] -->  Mikä kaunis päivä rostoille!
	[ run_Latn ] -->  Mbega umusi mwiza wo kurya umutobe!
	[ arb_Arab ] -->  يا له من يوم جميل لتناول الجبن
	[ acq_Arab ] -->  يوم جميل لخبز!


#Modelo para Hacer Resumenes ( Summarization )

In [10]:
#@title Cargar Modelo Summarization

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model_nameSum = "csebuetnlp/mT5_multilingual_XLSum"
tokenizerSum = AutoTokenizer.from_pretrained(model_nameSum)
modelSum = AutoModelForSeq2SeqLM.from_pretrained(model_nameSum)

def generate_summary(article_text):
    input_ids = tokenizerSum(
        [WHITESPACE_HANDLER(article_text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = modelSum.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizerSum.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary

print("\nModelo ", model_nameSum, " cargado.")    

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(



Modelo  csebuetnlp/mT5_multilingual_XLSum  cargado.


In [11]:
#@title Probar Modelo Summartizaton

text_to_summ = "Un modelo transformer es una red neuronal que aprende contexto y, por lo tanto, significado mediante el seguimiento de relaciones en datos secuenciales como las palabras de esta oración. Los modelos transformer aplican un conjunto en evolución de técnicas matemáticas, llamadas atención o atención propia, para detectar formas sutiles en que los elementos de datos en una serie se influencian y dependen entre sí. Estos transformers son una de las clases más nuevas y potentes de modelos inventados hasta la fecha. Están impulsando una ola de avances en machine learning que algunos han apodado como la «IA de transformer»." #@param {type:"string"}

print("> ", textwrap.fill(text_to_summ, 100))

print(" -->", generate_summary(text_to_summ) )

>  Un modelo transformer es una red neuronal que aprende contexto y, por lo tanto, significado mediante
el seguimiento de relaciones en datos secuenciales como las palabras de esta oración. Los modelos
transformer aplican un conjunto en evolución de técnicas matemáticas, llamadas atención o atención
propia, para detectar formas sutiles en que los elementos de datos en una serie se influencian y
dependen entre sí. Estos transformers son una de las clases más nuevas y potentes de modelos
inventados hasta la fecha. Están impulsando una ola de avances en machine learning que algunos han
apodado como la «IA de transformer».
 --> Transformers, una nueva clase de modelos matemáticos, están revolucionando la industria de la computación.


# Modelo Texto-To-Speech (TTS)

In [12]:
#@title Cargar Modelo TTS

nombreModeloTTS = "facebook/tts_transformer-es-css10"

TTS_models, TTS_cfg, TTS_task = load_model_ensemble_and_task_from_hf_hub(
    nombreModeloTTS,
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
TTS_model = TTS_models[0]
if torch.cuda.is_available():
  TTS_model = TTS_model.cuda()
TTSHubInterface.update_cfg_with_data_cfg(TTS_cfg, TTS_task.data_cfg)
TTS_generator = TTS_task.build_generator([TTS_model], TTS_cfg)


def speak(text):
  sample = TTSHubInterface.get_model_input(TTS_task, text)
  if torch.cuda.is_available():
    sample['net_input']['src_tokens'] = sample['net_input']['src_tokens'].cuda()
    sample['net_input']['src_lengths'] = sample['net_input']['src_lengths'].cuda()
    sample['speaker'] = sample['speaker'].cuda()
  wav, rate = TTSHubInterface.get_prediction(TTS_task, TTS_model, TTS_generator, sample)
  #ipd.Audio(wav, rate=rate)
  
  display( Audio(wav.cpu(), rate=rate, autoplay=True) )

print("\nModelo ", nombreModeloTTS, " cargado.")  

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]


Modelo  facebook/tts_transformer-es-css10  cargado.


In [13]:
#@title Probar Modelo TTS 1

text_to_wav = "la la la estoy cantando!! lara lara lala" #@param {type:"string"} 

print(">", text_to_wav)

# ejecuta el modelo 
speak(text_to_wav)


> la la la estoy cantando!! lara lara lala


In [14]:
#@title Probar Modelo TTS 2

text_to_wav = "San Martín cabalgaba en su caballo blanco cuando ataco al enemigo." #@param {type:"string"}

print(">", text_to_wav)

# ejecuta el modelo 
speak(text_to_wav)


> San Martín cabalgaba en su caballo blanco cuando ataco al enemigo.


# Probar Modelos combinados

In [16]:
#@title Probar Todos los Modelos 

texto_completo = "Podemos empezar con un castillo poco conocido pero realmente impresionante. Se trata del Nido de golondrina, ubicado cerca de Yalta, en la pen\xEDnsula de Crimea.   Quiz\xE1 no cuente con tantos siglos de historia como el resto, pues fue terminado en 1921, pero su ubicaci\xF3n impresiona, y mucho, a unos 40 metros de altura en una roca frente al mar negro. Asimismo, su construcci\xF3n con claros motivos neog\xF3ticos, hacen que este castillo parezca una hermosa fortaleza de cuento que bien podr\xEDa ser el escenario de historias como la de la Bella Durmiente o Blancanieves.  No hace falta irse muy lejos para poder ver un castillo de cuento, en Espa\xF1a encontramos varios, como por ejemplo el Alc\xE1zar de Segovia. Se trata de uno de los monumentos m\xE1s destacados de la ciudad y del pa\xEDs. Ubicado sobre un cerro con vistas impresionantes, este castillo es el resultado de los gustos arquitect\xF3nico de distintos monarcas. Un lugar que adem\xE1s de sorprender por su encantadora belleza, relata historias palaciegas de lo m\xE1s variadas a todo aquel que lo visita. Exterior de ensue\xF1o e interiores llenos de lujo y belleza, decorados por pintores y artistas de diversos puntos del mundo. Una joya arquitect\xF3nica que no defrauda ni lo m\xE1s m\xEDnimo." #@param {type:"string"}

pregunta = "que no defrauda?" #@param {type:"string"}

lenguaje_traducir = "zho_Hans" #@param ["ace_Arab",  "ace_Latn",  "acm_Arab",  "acq_Arab",  "aeb_Arab",  "afr_Latn",  "ajp_Arab",  "aka_Latn",  "amh_Ethi",  "apc_Arab",  "arb_Arab",  "ars_Arab",  "ary_Arab",  "arz_Arab",  "asm_Beng",  "ast_Latn",  "awa_Deva",  "ayr_Latn",  "azb_Arab",  "azj_Latn",  "bak_Cyrl",  "bam_Latn",  "ban_Latn",  "bel_Cyrl",  "bem_Latn",  "ben_Beng",  "bho_Deva",  "bjn_Arab",  "bjn_Latn",  "bod_Tibt",  "bos_Latn",  "bug_Latn",  "bul_Cyrl",  "cat_Latn",  "ceb_Latn",  "ces_Latn",  "cjk_Latn",  "ckb_Arab",  "crh_Latn",  "cym_Latn",  "dan_Latn",  "deu_Latn",  "dik_Latn",  "dyu_Latn",  "dzo_Tibt",  "ell_Grek",  "eng_Latn",  "epo_Latn",  "est_Latn",  "eus_Latn",  "ewe_Latn",  "fao_Latn",  "pes_Arab",  "fij_Latn",  "fin_Latn",  "fon_Latn",  "fra_Latn",  "fur_Latn",  "fuv_Latn",  "gla_Latn",  "gle_Latn",  "glg_Latn",  "grn_Latn",  "guj_Gujr",  "hat_Latn",  "hau_Latn",  "heb_Hebr",  "hin_Deva",  "hne_Deva",  "hrv_Latn",  "hun_Latn",  "hye_Armn",  "ibo_Latn",  "ilo_Latn",  "ind_Latn",  "isl_Latn",  "ita_Latn",  "jav_Latn",  "jpn_Jpan",  "kab_Latn",  "kac_Latn",  "kam_Latn",  "kan_Knda",  "kas_Arab",  "kas_Deva",  "kat_Geor",  "knc_Arab",  "knc_Latn",  "kaz_Cyrl",  "kbp_Latn",  "kea_Latn",  "khm_Khmr",  "kik_Latn",  "kin_Latn",  "kir_Cyrl",  "kmb_Latn",  "kon_Latn",  "kor_Hang",  "kmr_Latn",  "lao_Laoo",  "lvs_Latn",  "lij_Latn",  "lim_Latn",  "lin_Latn",  "lit_Latn",  "lmo_Latn",  "ltg_Latn",  "ltz_Latn",  "lua_Latn",  "lug_Latn",  "luo_Latn",  "lus_Latn",  "mag_Deva",  "mai_Deva",  "mal_Mlym",  "mar_Deva",  "min_Latn",  "mkd_Cyrl",  "plt_Latn",  "mlt_Latn",  "mni_Beng",  "khk_Cyrl",  "mos_Latn",  "mri_Latn",  "zsm_Latn",  "mya_Mymr",  "nld_Latn",  "nno_Latn",  "nob_Latn",  "npi_Deva",  "nso_Latn",  "nus_Latn",  "nya_Latn",  "oci_Latn",  "gaz_Latn",  "ory_Orya",  "pag_Latn",  "pan_Guru",  "pap_Latn",  "pol_Latn",  "por_Latn",  "prs_Arab",  "pbt_Arab",  "quy_Latn",  "ron_Latn",  "run_Latn",  "rus_Cyrl",  "sag_Latn",  "san_Deva",  "sat_Beng",  "scn_Latn",  "shn_Mymr",  "sin_Sinh",  "slk_Latn",  "slv_Latn",  "smo_Latn",  "sna_Latn",  "snd_Arab",  "som_Latn",  "sot_Latn",  "spa_Latn",  "als_Latn",  "srd_Latn",  "srp_Cyrl",  "ssw_Latn",  "sun_Latn",  "swe_Latn",  "swh_Latn",  "szl_Latn",  "tam_Taml",  "tat_Cyrl",  "tel_Telu",  "tgk_Cyrl",  "tgl_Latn",  "tha_Thai",  "tir_Ethi",  "taq_Latn",  "taq_Tfng",  "tpi_Latn",  "tsn_Latn",  "tso_Latn",  "tuk_Latn",  "tum_Latn",  "tur_Latn",  "twi_Latn",  "tzm_Tfng",  "uig_Arab",  "ukr_Cyrl",  "umb_Latn",  "urd_Arab",  "uzn_Latn",  "vec_Latn",  "vie_Latn",  "war_Latn",  "wol_Latn",  "xho_Latn",  "ydd_Hebr",  "yor_Latn",  "yue_Hant",  "zho_Hans",  "zho_Hant",  "zul_Latn" ]

print("\n")
print("> ", textwrap.fill(texto_completo, 100))

# resume
txt_resumido = generate_summary(texto_completo)
print("\t  RESUMEN: ", textwrap.fill(txt_resumido, 100))

# traduce resumen
resTrad = translate(txt_resumido, "spa_Latn", lenguaje_traducir)
print("\t\t (", textwrap.fill(resTrad, 100), ")")

# habla (solo resumido)
speak(txt_resumido)


print("")
print("\t", pregunta, " (con texto completo)")

# procesa pregunta por modelo Q&A para texto completo
res = responder_pregunta(pregunta, texto_completo)
print("\t  -->", res)
  
# traduce
q_resTrad = translate(pregunta, "spa_Latn", lenguaje_traducir)
r_resTrad = translate(res, "spa_Latn", lenguaje_traducir)

print("\t\t\t(", q_resTrad + " --> " + r_resTrad, ")")

# habla
speak( pregunta + " " + res)

print("")
print("\t", pregunta, " (con resumen)")

# procesa pregunta por modelo Q&A para resumen
res = responder_pregunta(pregunta, txt_resumido)

print("\t  -->", res)

# traduce
#q_resTrad = translate(pregunta, "spa_Latn", lenguaje_traducir)
r_resTrad = translate(res, "spa_Latn", lenguaje_traducir)

print("\t\t (", q_resTrad + " --> " + r_resTrad, ")")

# habla
speak( pregunta + " " + res)




>  Podemos empezar con un castillo poco conocido pero realmente impresionante. Se trata del Nido de
golondrina, ubicado cerca de Yalta, en la península de Crimea.   Quizá no cuente con tantos siglos
de historia como el resto, pues fue terminado en 1921, pero su ubicación impresiona, y mucho, a unos
40 metros de altura en una roca frente al mar negro. Asimismo, su construcción con claros motivos
neogóticos, hacen que este castillo parezca una hermosa fortaleza de cuento que bien podría ser el
escenario de historias como la de la Bella Durmiente o Blancanieves.  No hace falta irse muy lejos
para poder ver un castillo de cuento, en España encontramos varios, como por ejemplo el Alcázar de
Segovia. Se trata de uno de los monumentos más destacados de la ciudad y del país. Ubicado sobre un
cerro con vistas impresionantes, este castillo es el resultado de los gustos arquitectónico de
distintos monarcas. Un lugar que además de sorprender por su encantadora belleza, relata historias
palaciega


	 que no defrauda?  (con texto completo)
	  --> Una joya arquitectónica que no defrauda ni lo más mínimo
			( 他不会让人失望吗? --> 建筑宝石,不会让任何一个人失望 )



	 que no defrauda?  (con resumen)
	  --> una joya arquitectónica
		 ( 他不会让人失望吗? --> 一个建筑宝石 )
